# Apply

::::::{attention}
This page shows a preview of the assignment. Please fork and clone the assignment to work on it locally from [GitHub](https://github.com/CIEM5000-2025/practice-assignments)

After the workshop, the solution will be added to this preview and to the [GitHub-repository](https://github.com/CIEM5000-2025/practice-assignments)
::::::

In this notebook you will work on a homework assignment involving a Vierendeel frame.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

```{custom_download_link} ./Workshop_1_Apply_stripped.ipynb
:text: ".ipynb"
:replace_default: "True"
```

```{custom_download_link} ./Workshop_1_Apply.md
:text: ".md:myst"
:replace_default: "False"
```

```{custom_download_link} https://github.com/CIEM5000-2025/practice-assignments
:text: "All files practice assignments"
:replace_default: "False"
```

## Vierendeel frame

```{figure} https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png
:align: center
:width: 400
```

With:

- $h = 1$
- $b = 1$
- $EI_r = 10000$
- $EI_k = 1000$
- $EA  = 1\cdot 10^{10}$
- $H = 100$

In the first half of this course last quarter, you have learned that the deformation of Vierendeel frames (an example of which is shown above) can be obtained in a simplified way by assuming the global deformation can be described by a shear beam with equivalent stiffness given by:

$$
k = \frac{24}{h\left(\displaystyle\frac{h}{EI_k}+\frac{b}{EI_r}\right)}
$$

```{exercise-start} Workshop 1 - Apply
:label: exercise_ws_1
:nonumber: true

Now that you have the tools to solve the original frame problem using the Matrix Method, your task in this assignment is to investigate the validity of this equivalent shear beam model.

Note that the checks only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (Tip: refer back to what you did in the `constrain` function).

Once you have a solution, use SymPy / Maple / pen and paper to solve a shear beam problem with the equivalent stiffness given above (It is very similar to the simple extension problem above) and compare the horizontal displacement at the point of application of $H$ for the two models.

Investigate how the two models compare for different values of $EA$, ranging from very small (*e.g.* $1\cdot 10^{-5}$) to very large (*e.g.* $1\cdot10^{10}$). What explains the behavior you observe?
```

In [2]:
import matplotlib as plt
import numpy as np
#sys.path.insert(1, '/matrixmethod_solution')
import matrixmethod_solution as mm
%config InlineBackend.figure_formats = ['svg']

In [3]:
import matplotlib as plt
import numpy as np
import matrixmethod_solution as mm
%config InlineBackend.figure_formats = ['svg']

In [5]:
import numpy as np
import matplotlib as plt
#import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [17]:
mm.Node.clear()
mm.Element.clear()

In [18]:
h = 1
b = 1
EI_r = 10000
EI_k = 1000
EA = 1e10
H = 100

In [ ]:
nodes = []

nodes.append(mm.Node(0, 0))
nodes.append(mm.Node(b, 0))
nodes.append(mm.Node(b, -h))
nodes.append(mm.Node(0, -h))

print(nodes)

elems = []

elems.append(mm.Element(nodes[0], nodes[1]))
elems.append(mm.Element(nodes[1], nodes[2]))
elems.append(mm.Element(nodes[2], nodes[3]))
elems.append(mm.Element(nodes[0], nodes[3]))

beams = {}
columns = {}
beams['EI'] = EI_r
beams['EA'] = EA
columns['EI'] = EI_k
columns['EA'] = EA

elems[0].set_section(beams)
elems[2].set_section(beams)
elems[1].set_section(columns)
elems[3].set_section(columns)

for elem in elems:
    print(elem)

con = mm.Constrainer()

con.fix_dof(nodes[0], 0)
con.fix_dof(nodes[0], 1)
con.fix_dof(nodes[1], 1)

nodes[3].add_load([H, 0, 0])

    


[<matrixmethod_solution.node.Node object at 0x000002410C8BCF80>, <matrixmethod_solution.node.Node object at 0x000002410C8BF710>, <matrixmethod_solution.node.Node object at 0x000002410BD3F590>, <matrixmethod_solution.node.Node object at 0x000002410C8BE600>]
Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
Element connecting:
node #1:
 This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=-1,
 - degrees of freedom=[6, 7, 8],
 - load vector=[0. 0. 0.]
Element connecting:
node #1:
 This node has:
 - x coordinate=1,
 - z coordinate=-1,
 - degrees of freedom=[6, 7, 8],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=0,
 - z coordin

In [23]:
global_k = np.zeros((3*len(nodes), 3*len(nodes)))
global_f = np.zeros(3*len(nodes))

for elem in elems:
    elmat = elem.stiffness()
    idofs = elem.global_dofs()

    global_k[np.ix_(idofs, idofs)] += elmat

for node in nodes:
    global_f[node.dofs] += node.p


Kff, Ff = con.constrain(global_k, global_f)
u = np.matmul(np.linalg.inv(Kff), Ff)
print(u)

[-4.16667982e-04  4.99999539e-09 -4.16665674e-04  4.58333833e-03
  4.99999807e-09 -4.16675352e-04  4.58334333e-03 -4.99999806e-09
 -4.16677659e-04]


In [ ]:
#provided in case you want to solve the shear beam problem using SymPy
import sympy as sym
x, k, L, H = sym.symbols('x, k, L, H')
w = sym.Function('w')

ODE_shear = #YOUR CODE HERE

```{exercise-end}
```